# 1 - Importing libraries

In [25]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql.types import NumericType, StructType, Row
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import numpy as np
import basedosdados as bd
import pandas as pd

## 1.1 - Setuping ETL logging

In [7]:
from loggings.logging_setup import LoggerSetup

In [8]:
etl_logger = LoggerSetup.get_etl_logger()

# 2 - Initializing Spark Session

In [9]:
from utils.init_spark_session import SparkSessionManager

SPARK_CONFIG: dict = {
    "spark.executor.memory": "4g",
    "spark.executor.cores": "2",
    "spark.driver.extraClassPath": "C:\\mysql-connector-j-8.4.0.jar",
    "spark.driver.memory": "4g",
    "spark.sql.shuffle.partitions": "200",
    "spark.default.parallelism": "100"
}

spark = SparkSessionManager.get_spark_session(
    app_name="ETL Process", config=SPARK_CONFIG
    )

# 3 - Extracting

## 3.1 - Scraping file

In [11]:
from scraping_file.arrecadacao_estado_scrap_file import FileHandlerArrecadacaoEstado
import os

download_dir = r"C:\RevenueByState\data"

scrap = FileHandlerArrecadacaoEstado(download_dir)

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

paths = os.listdir(r"C:\RevenueByState")
name_path = download_dir[download_dir.rfind('\\') + 1:]

if name_path not in paths or "arrecadacao-estado.csv" not in os.listdir(download_dir):
    scrap.download_file()
    etl_logger.info("File downloaded with success")
else:
    etl_logger.warning("File already exists")

2024-08-23 10:16:38,709 - INFO - File downloaded with success


## 3.2 - Getting another resources (population by states) from https://basedosdados.org/

In [12]:
def get_dataframe_from_basedosdados(dataset_id: str, table_id: str, billing_project_id: str) -> pd.DataFrame:
    """
    Read dataframe from basedosdados

    Parameters
    ----------
    dataset_id: str
        The dataset id where the table is in
    table_id: str
        To identify our table
    billing_project_id: str
        Our billing project 
        
    Returns
    -------
    pd.DataFrame
        A pandas DataFrame with our resource
    """

    return bd.read_table(
        dataset_id=dataset_id,
        table_id=table_id,
        billing_project_id=billing_project_id
    )

In [13]:
population_states_df = get_dataframe_from_basedosdados(
    dataset_id="br_ibge_populacao", table_id="uf", billing_project_id="revenue-etl"
    )

Downloading: 100%|██████████| 837/837 [00:00<00:00, 2126.11rows/s]


In [15]:
inflation_df = get_dataframe_from_basedosdados(
    dataset_id="br_ibge_inpc", table_id="mes_brasil", billing_project_id="revenue-etl"
) 

Downloading: 100%|██████████| 540/540 [00:00<00:00, 1084.27rows/s]


In [33]:
# changing np.nan to None (for Spark) 
population_states_df = population_states_df.replace([np.nan], [None])
inflation_df = inflation_df.replace([np.nan], [None])

In [35]:
population_states_df

,sigla_uf,ano,populacao,populacao_economicamente_ativa
0,AC,1991,417102,None
1,AL,1991,2512658,None
2,AM,1991,2102766,None
3,AP,1991,289035,None
4,BA,1991,11867336,None
...,...,...,...,...
832,RS,2021,11466630,9533
833,SC,2021,7338473,5947
834,SE,2021,2338474,1873
835,SP,2021,46649132,38573


In [36]:
inflation_df

,ano,mes,indice,variacao_mensal,variacao_trimestral,variacao_semestral,variacao_anual,variacao_doze_meses
0,1979,3,0.0,None,None,None,None,None
1,1979,4,0.0,3.45,None,None,None,None
2,1979,5,0.0,1.76,None,None,None,None
3,1979,6,0.0,3.0,8.43,None,None,None
4,1979,7,0.0,5.36,10.43,None,None,None
...,...,...,...,...,...,...,...,...
535,2023,10,6909.79,0.12,0.43,0.6,3.04,4.14
536,2023,11,6916.7,0.1,0.33,0.34,3.14,3.85
537,2023,11,6916.7,0.1,0.33,0.34,3.14,3.85
538,2023,12,6954.74,0.55,0.77,0.99,3.71,3.71


## 3.3 - Reading resources

### 3.3.1 - Listing file paths

In [37]:
data = {
    "arrecadacao_estado": "C:\RevenueByState\data\\arrecadacao-estado.csv"
}

### 3.3.2 - Defining Schemas

In [38]:
# Defining the schema (translating columns from Portuguese to English)
collection_state_schema = StructType([
    StructField("year", IntegerType(), nullable=True),
    StructField("month", StringType(), nullable=True),
    StructField("state", StringType(), nullable=True),
    StructField("import_tax", DoubleType(), nullable=True),
    StructField("export_tax", DoubleType(), nullable=True),
    StructField("ipi_tobacco", DoubleType(), nullable=True),
    StructField("ipi_beverages", DoubleType(), nullable=True),
    StructField("ipi_auto", DoubleType(), nullable=True),
    StructField("ipi_linked_imports", DoubleType(), nullable=True),
    StructField("ipi_others", DoubleType(), nullable=True),
    StructField("income_tax_individual", DoubleType(), nullable=True),
    StructField("income_tax_financial", DoubleType(), nullable=True),
    StructField("income_tax_other", DoubleType(), nullable=True),
    StructField("withholding_tax_employment", DoubleType(), nullable=True),
    StructField("withholding_tax_capital", DoubleType(), nullable=True),
    StructField("withholding_tax_remittances", DoubleType(), nullable=True),
    StructField("withholding_tax_other", DoubleType(), nullable=True),
    StructField("tax_financial_operations", DoubleType(), nullable=True),
    StructField("rural_land_tax", DoubleType(), nullable=True),
    StructField("provisional_tax_transactions", DoubleType(), nullable=True),
    StructField("provisional_contribution_transactions", DoubleType(), nullable=True),
    StructField("cofins", DoubleType(), nullable=True),
    StructField("cofins_financial", DoubleType(), nullable=True),
    StructField("cofins_other", DoubleType(), nullable=True),
    StructField("contribution_social_integration", DoubleType(), nullable=True),
    StructField("contribution_social_integration_financial", DoubleType(), nullable=True),
    StructField("contribution_social_integration_other", DoubleType(), nullable=True),
    StructField("social_contribution_net_profit", DoubleType(), nullable=True),
    StructField("social_contribution_net_profit_financial", DoubleType(), nullable=True),
    StructField("social_contribution_net_profit_other", DoubleType(), nullable=True),
    StructField("intervention_economic_domain_non_deductible_fuels", DoubleType(), nullable=True),
    StructField("intervention_economic_domain_fuels", DoubleType(), nullable=True),
    StructField("contribution_security_plan_public_servants", DoubleType(), nullable=True),
    StructField("contributions_security_plan_public_servants", DoubleType(), nullable=True),
    StructField("contributions_special_fund_inspection_activities", DoubleType(), nullable=True),
    StructField("fiscal_recovery_program", DoubleType(), nullable=True),
    StructField("installment_payment_program", DoubleType(), nullable=True),
    StructField("withholding_tax_law_article_30", DoubleType(), nullable=True),
    StructField("unified_payment", DoubleType(), nullable=True),
    StructField("other_administered_revenues", DoubleType(), nullable=True),
    StructField("other_revenues", DoubleType(), nullable=True),
    StructField("social_security_revenue", DoubleType(), nullable=True),
    StructField("social_security_revenue_individual_contributors", DoubleType(), nullable=True),
    StructField("social_security_revenue_other_sources", DoubleType(), nullable=True),
    StructField("administered_by_other_agencies", DoubleType(), nullable=True)
])

In [39]:
population_states_schema = StructType([
    StructField(name="state_uf", dataType=StringType(), nullable=True),
    StructField(name="year", dataType=IntegerType(), nullable=True),
    StructField(name="population_state", dataType=IntegerType(), nullable=True),
    StructField(name="economically_active_population", dataType=IntegerType(), nullable=True)
])

In [40]:
inflation_schema = StructType([
    StructField("year", IntegerType(), nullable=True),
    StructField("month", IntegerType(), nullable=True),
    StructField("index", FloatType(), nullable=True),
    StructField("monthly_variation", FloatType(), nullable=True),
    StructField("quarterly_variation", FloatType(), nullable=True),
    StructField("semiannual_variation", FloatType(), nullable=True),
    StructField("annual_variation", FloatType(), nullable=True),
    StructField("twelve_months_variation", FloatType(), nullable=True)
])

### 3.3.3 - Reading all files

In [41]:
def read_file(file_name: str, format: str, schema: StructType, data: dict, delimiter: str = None) -> DataFrame:
    try:
        if file_name not in data:
            raise ValueError(f"File '{file_name}' not found in the provided data dictionary.")

        reader = spark.read \
            .format(format) \
            .option("header", "true") \
            .schema(schema)

        if delimiter:
            reader = reader.option("delimiter", delimiter)

        df = reader.load(data.get(file_name))

        return df
    except Exception as e:
        etl_logger.error(f"Error occurred while loading DataFrame for file '{file_name}': {e}")
        return None

In [42]:
collection_state_df = read_file(
    file_name="arrecadacao_estado",
    format="csv",
    schema=collection_state_schema,
    data=data,
    delimiter=";"
    )

In [43]:
def create_spark_dataframe_from_pandas(df: pd.DataFrame, schema=StructType) -> DataFrame:
    try:
        return spark.createDataFrame(df, schema)
    except Exception as e:
        etl_logger.error(f"Error occurred while loading DataFrame: {e}")
        return None

In [44]:
population_states_df = create_spark_dataframe_from_pandas(df=population_states_df, schema=population_states_schema)
inflation_df = create_spark_dataframe_from_pandas(df=inflation_df, schema=inflation_schema)

In [45]:
collection_state_df.printSchema()
population_states_df.printSchema()
inflation_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- state: string (nullable = true)
 |-- import_tax: double (nullable = true)
 |-- export_tax: double (nullable = true)
 |-- ipi_tobacco: double (nullable = true)
 |-- ipi_beverages: double (nullable = true)
 |-- ipi_auto: double (nullable = true)
 |-- ipi_linked_imports: double (nullable = true)
 |-- ipi_others: double (nullable = true)
 |-- income_tax_individual: double (nullable = true)
 |-- income_tax_financial: double (nullable = true)
 |-- income_tax_other: double (nullable = true)
 |-- withholding_tax_employment: double (nullable = true)
 |-- withholding_tax_capital: double (nullable = true)
 |-- withholding_tax_remittances: double (nullable = true)
 |-- withholding_tax_other: double (nullable = true)
 |-- tax_financial_operations: double (nullable = true)
 |-- rural_land_tax: double (nullable = true)
 |-- provisional_tax_transactions: double (nullable = true)
 |-- provisional_contribution_transactio

In [46]:
collection_state_df.show()

+----+-------+-----+-----------+----------+-----------+-------------+-----------+------------------+-----------+---------------------+--------------------+----------------+--------------------------+-----------------------+---------------------------+---------------------+------------------------+--------------+----------------------------+-------------------------------------+------------+----------------+------------+-------------------------------+-----------------------------------------+-------------------------------------+------------------------------+----------------------------------------+------------------------------------+-------------------------------------------------+----------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------------+-----------------------+---------------------------+------------------------------+---------------+---------------------------+--------

In [47]:
collection_state_df = collection_state_df.cache()
population_states_df = population_states_df.cache()
inflation_df = inflation_df.cache()

# 4 - Transforming

## 4.1 - Validate and Cleanse Data

### 4.1.1 - Joining tables (population_states and collection_state)

In [18]:
# alias for the tables
collection_state_df = collection_state_df.alias("cs")
population_states_df = population_states_df.alias("ps")

In [19]:
# joinin and selecting columns
collection_state_df = collection_state_df.join(
    population_states_df,
    (population_states_df.year == collection_state_df.year)
    &
    (collection_state_df.state == population_states_df.state_uf),
    "left"
).select(
    "cs.*",
    "ps.population_state",
    "ps.economically_active_population"
)

### 4.1.2 - Solving ASCII problem

In [20]:
# we have a non ASCII character (ç), so we need to translate
special_char = "�"

correct_char = "c"

collection_state_df = collection_state_df.withColumn("month", translate(col("month"), special_char, correct_char))

In [21]:
# changing month names from Portuguese to English
collection_state_df = collection_state_df.withColumn(
    "month",
    when(col("month") == "Janeiro", "january")
    .when(col("month") == "Fevereiro", "february")
    .when(col("month") == "Marco", "march")
    .when(col("month") == "Abril", "april")
    .when(col("month") == "Maio", "may")
    .when(col("month") == "Junho", "june")
    .when(col("month") == "Julho", "july")
    .when(col("month") == "Agosto", "august")
    .when(col("month") == "Setembro", "september")
    .when(col("month") == "Outubro", "october")
    .when(col("month") == "Novembro", "november")
    .when(col("month") == "Dezembro", "december")
    .otherwise(col("month"))
)

In [22]:
collection_state_df.show()

+----+---------+-----+------------+----------+------------+-------------+-----------+------------------+-----------+---------------------+--------------------+----------------+--------------------------+-----------------------+---------------------------+---------------------+------------------------+--------------+----------------------------+-------------------------------------+------+----------------+------------+-------------------------------+-----------------------------------------+-------------------------------------+------------------------------+----------------------------------------+------------------------------------+-------------------------------------------------+----------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------------+-----------------------+---------------------------+------------------------------+---------------+---------------------------+----------

### 4.1.3 - Checking count grouped by month

In [23]:
months_counts_names = collection_state_df.groupBy("month").count()
months_counts_names.show()

+---------+-----+
|    month|count|
+---------+-----+
|  october|  648|
|      may|  648|
|september|  648|
|   august|  648|
|    april|  675|
| november|  648|
|     july|  648|
|  january|  675|
| february|  675|
|    march|  675|
|     june|  648|
| december|  648|
+---------+-----+



### 4.1.4 - Adding month in numeric representation

In [24]:
months = {
    "january": 1,
    "february": 2,
    "march": 3,
    "april": 4,
    "may": 5,
    "june": 6,
    "july": 7,
    "august": 8,
    "september": 9,
    "october": 10,
    "november": 11,
    "december": 12
}

In [25]:
collection_state_df = collection_state_df.withColumn(
    "month_numeric",
    when(col("month") == "january", lit(months["january"]))
    .when(col("month") == "february", lit(months["february"]))
    .when(col("month") == "march", lit(months["march"]))
    .when(col("month") == "april", lit(months["april"]))
    .when(col("month") == "may", lit(months["may"]))
    .when(col("month") == "june", lit(months["june"]))
    .when(col("month") == "july", lit(months["july"]))
    .when(col("month") == "august", lit(months["august"]))
    .when(col("month") == "september", lit(months["september"]))
    .when(col("month") == "october", lit(months["october"]))
    .when(col("month") == "november", lit(months["november"]))
    .when(col("month") == "december", lit(months["december"]))
    .otherwise(None)
)

### 4.1.5 - Checking count grouped by month in numeric representation

In [26]:
numeric_month_count = collection_state_df.groupBy("month_numeric").count()
numeric_month_count.show()

+-------------+-----+
|month_numeric|count|
+-------------+-----+
|           12|  648|
|            1|  675|
|            6|  648|
|            3|  675|
|            5|  648|
|            9|  648|
|            4|  675|
|            8|  648|
|            7|  648|
|           10|  648|
|           11|  648|
|            2|  675|
+-------------+-----+



### 4.1.6 - Joining tables (collection_state and inflation_df)

#### 4.1.6.1 - Tables alias

In [27]:
collection_state_df = collection_state_df.alias("cs")
inflation_df = inflation_df.alias("inf")

### 4.1.6.1 - Joining both tables with alias

In [28]:
collection_state_df = collection_state_df.join(
    inflation_df,
    (collection_state_df.year == inflation_df.year)
    &
    (collection_state_df.month_numeric == inflation_df.month),
    "left"
).select(
    "cs.*",
    "inf.index",
    "inf.monthly_variation",
    "inf.quarterly_variation",
    "inf.semiannual_variation",
    "inf.annual_variation"
)

In [29]:
collection_state_df.show()

+----+-------+-----+-----------+----------+-----------+-------------+-----------+------------------+-----------+---------------------+--------------------+----------------+--------------------------+-----------------------+---------------------------+---------------------+------------------------+--------------+----------------------------+-------------------------------------+------------+----------------+------------+-------------------------------+-----------------------------------------+-------------------------------------+------------------------------+----------------------------------------+------------------------------------+-------------------------------------------------+----------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------------+-----------------------+---------------------------+------------------------------+---------------+---------------------------+--------

### 4.1.7 - Removing duplicates

In [30]:
collection_state_df.dropDuplicates()

DataFrame[year: int, month: string, state: string, import_tax: double, export_tax: double, ipi_tobacco: double, ipi_beverages: double, ipi_auto: double, ipi_linked_imports: double, ipi_others: double, income_tax_individual: double, income_tax_financial: double, income_tax_other: double, withholding_tax_employment: double, withholding_tax_capital: double, withholding_tax_remittances: double, withholding_tax_other: double, tax_financial_operations: double, rural_land_tax: double, provisional_tax_transactions: double, provisional_contribution_transactions: double, cofins: double, cofins_financial: double, cofins_other: double, contribution_social_integration: double, contribution_social_integration_financial: double, contribution_social_integration_other: double, social_contribution_net_profit: double, social_contribution_net_profit_financial: double, social_contribution_net_profit_other: double, intervention_economic_domain_non_deductible_fuels: double, intervention_economic_domain_fuels

### 4.1.8 - Handling missing values 

In [31]:
# string(will be removed), numeric(flag 0)
def fill_values_na_with_zero_or_drop(df: DataFrame) -> DataFrame:
    for col, dtype in df.dtypes:
        if isinstance(df.schema[col].dataType, NumericType):
            df = df.fillna(0, subset=[col])
        elif isinstance(df.schema[col].dataType, StringType):
            df = df.dropna(subset=[col])
    return df

In [32]:
collection_state_df = fill_values_na_with_zero_or_drop(df = collection_state_df)

## 4.2 - Adding some columns

### 4.2.1 - Regions in Brazil

In [33]:
regions_uf = {
    "north": ("AC", "AP", "AM", "PA", "RO", "RR", "TO"),
    "north_east": ("AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"),
    "midwest": ("DF", "GO", "MT", "MS"),
    "south": ("PR", "RS", "SC"),
    "south_east": ("ES", "MG", "RJ", "SP")
}

In [34]:
# function to add column "region" based on regions_uf
def add_region_column(df: DataFrame, regions_uf: dict) -> DataFrame:
    df = df.withColumn(
        "region",
        when(col("state").isin(list(regions_uf["north"])), "north")
        .when(col("state").isin(list(regions_uf["north_east"])), "north_east")
        .when(col("state").isin(list(regions_uf["midwest"])), "midwest")
        .when(col("state").isin(list(regions_uf["south"])), "south")
        .when(col("state").isin(list(regions_uf["south_east"])), "south_east")
        .otherwise("unknown")
    )
    return df

In [35]:
collection_state_df = add_region_column(df=collection_state_df, regions_uf=regions_uf)

### 4.2.2 - Adding 'id' column

In [87]:
windowSpec = Window.orderBy("year")
collection_state_df = collection_state_df.withColumn("id", F.row_number().over(windowSpec))

# Reordering columns to have 'id' as the first column
id_first_columns = ['id'] + [col for col in collection_state_df.columns if col != 'id']

### 4.2.3 - Moving region column to be near of the state column

In [88]:
regions = id_first_columns[-1]
id_first_columns.insert(4, regions)
id_first_columns.pop() # to remove the last element (region)

collection_state_final_df = collection_state_df.select(id_first_columns)

In [90]:
collection_state_final_df.show()

+---+----+-------+-----+----------+-----------+----------+-----------+-------------+-----------+------------------+-----------+---------------------+--------------------+----------------+--------------------------+-----------------------+---------------------------+---------------------+------------------------+--------------+----------------------------+-------------------------------------+------------+----------------+------------+-------------------------------+-----------------------------------------+-------------------------------------+------------------------------+----------------------------------------+------------------------------------+-------------------------------------------------+----------------------------------+------------------------------------------+-------------------------------------------+------------------------------------------------+-----------------------+---------------------------+------------------------------+---------------+---------------------

# 5 - Loading

## 5.1 - Creating MySQL database

In [91]:
from script.create_database import DatabaseConnector
host = os.getenv('DB_HOST', 'localhost')
user = os.getenv('DB_USER', 'root')
password = os.getenv('DB_PASSWORD', '')

db_connector = DatabaseConnector(host=host, user=user, password=password)
    
try:
    db_connector.get_connection()
    db_connector.create_database("revenue_by_state")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    db_connector.close_connection()

2024-06-30 14:36:55,560 - INFO - Connected to MySQL server
2024-06-30 14:36:55,563 - INFO - Database 'revenue_by_state' created or already exists
2024-06-30 14:36:55,564 - INFO - MySQL connection is closed


## 5.2 - Defining url and properties

In [92]:
jdbc_url, jdbc_properties = db_connector.get_jdbc_url_and_properties()

## 5.3 - Writing on MySQL

In [93]:
table_name: str = "collection_state"
mode: str = "overwrite"

try:
    collection_state_final_df.write.jdbc(
        url=jdbc_url,
        table=table_name,
        mode=mode,
        properties=jdbc_properties
        )
    print("Loaded with success")
except Exception as e:
    print(f"Error: {e}")

Loaded with success
